In [1]:
import os
import platform
import sys

def get_project_root() -> str:
    system = platform.system()
    return r"D:\Users\tcshore\Documents\Projects\Tangrams\Data" if system == "Windows" else "/home/tshore/Projects/tangrams-restricted/Data"

infile_path = os.path.join(get_project_root(), "wordscores-inflected-small.tsv")
print("Will read file \"{}\".".format(infile_path), file=sys.stderr)

Will read file "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordscores-inflected-small.tsv".


In [2]:
import csv
import pandas as pd

RESULTS_FILE_CSV_DIALECT = csv.excel_tab
# NOTE: "category" dtype doesn't work with pandas-0.21.0 but does with pandas-0.21.1
__RESULTS_FILE_DTYPES = {"DYAD": "category", "ENTITY" : "category", "IS_TARGET": bool, "IS_OOV": bool,
				 "IS_INSTRUCTOR": bool, "SHAPE": "category", "ONLY_INSTRUCTOR": bool, "WEIGHT_BY_FREQ": bool}

def read_results_file(inpath: str, encoding: str) -> pd.DataFrame:
	print("Reading \"{}\" using encoding \"{}\".".format(inpath, encoding), file=sys.stderr)
	result = pd.read_csv(inpath, dialect=RESULTS_FILE_CSV_DIALECT, sep=RESULTS_FILE_CSV_DIALECT.delimiter,
						 float_precision="round_trip",
						 encoding=encoding, memory_map=True, dtype=__RESULTS_FILE_DTYPES)
	return result

cv_results = read_results_file(infile_path, "windows-1252")
print("Read {} cross-validation results for {} dyad(s).".format(cv_results.shape[0], cv_results["DYAD"].nunique()),
      file=sys.stderr)

Reading "D:\Users\tcshore\Documents\Projects\Tangrams\Data\wordscores-inflected-small.tsv" using encoding "windows-1252".
Read 67060 cross-validation results for 3 dyad(s).


In [3]:
OOV_LABEL = "__OOV__"
    
print("Replacing OOV words with label \"{}\".".format(OOV_LABEL), file=sys.stderr)
cv_results.loc[cv_results["IS_OOV"] == True, "WORD"] = OOV_LABEL

Replacing OOV words with label "__OOV__".


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Create vocab before splitting training and testing DFs so that the word feature set is stable
print("Fitting one-hot encoder for vocabulary of size {}.".format(cv_results["WORD"].nunique()), file=sys.stderr)

# https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
# integer encode
label_encoder = LabelEncoder()
vocab_labels = label_encoder.fit_transform(cv_results["WORD"])
cv_results["WORD_LABEL"] = vocab_labels
#print(vocab_labels)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
vocab_labels = vocab_labels.reshape(len(vocab_labels), 1)
onehot_encoder.fit(vocab_labels)
#assert onehot_encoder.n_values_ == len(vocab_words)
#vocab_onehot_encoded = onehot_encoder.fit_transform(vocab_labels)
#print(vocab_onehot_encoded)
# invert first example
#inverted = label_encoder.inverse_transform([np.argmax(vocab_onehot_encoded[0, :])])
#print(inverted)

Fitting one-hot encoder for vocabulary of size 249.


OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [5]:
from typing import Tuple

import numpy as np

def find_target_ref_rows(df: pd.DataFrame) -> pd.DataFrame:
	result = df.loc[df["IS_TARGET"] == True]
	result_row_count = result.shape[0]
	complement_row_count = df.loc[~df.index.isin(result.index)].shape[0]
	assert result_row_count + complement_row_count == df.shape[0]
	print("Found {} nontarget rows and {} target rows. Ratio: {}".format(complement_row_count, result_row_count,
																		 complement_row_count / float(
																			 result_row_count)), file=sys.stderr)
	return result

def split_training_testing(df: pd.DataFrame, test_set_size : int) -> Tuple[pd.DataFrame, pd.DataFrame]:
	dyad_ids = df["DYAD"].unique()
	training_set_size = len(dyad_ids) - test_set_size
	if training_set_size < 1:
		raise ValueError("Desired test set size is {} but only {} dyads found.".format(test_set_size, len(dyad_ids)))
	else:
		training_set_dyads = frozenset(np.random.choice(dyad_ids, training_set_size))
		print("Training set dyads: {}".format(sorted(training_set_dyads)), file=sys.stderr)
		training_set_idxs = df["DYAD"].isin(training_set_dyads)
		training_set = df.loc[training_set_idxs]
		test_set = df.loc[~training_set_idxs]
		test_set_dyads = frozenset(test_set["DYAD"].unique())
		print("Test set dyads: {}".format(sorted(test_set_dyads)), file=sys.stderr)

		assert not frozenset(training_set["DYAD"].unique()).intersection(frozenset(test_set_dyads))
		return training_set, test_set
    
# https://stackoverflow.com/a/47815400/1391325
cv_results.sort_values("TOKEN_SEQ_ORDINALITY", inplace=True)
training_df, test_df = split_training_testing(cv_results, 1)
# Only train on "true" referents
training_df = find_target_ref_rows(training_df)

Training set dyads: ['20', '22']
Test set dyads: ['21']
Found 40489 nontarget rows and 2131 target rows. Ratio: 19.0


In [90]:
from collections import defaultdict
from typing import DefaultDict, Iterator, List

import keras.utils

class DataGeneratorFactory(object):

	def __init__(self, onehot_encoder):
		self.onehot_encoder = onehot_encoder

	@property
	def input_feature_count(self) -> int:
		word_features = self.onehot_encoder.n_values_[0]
		return word_features + 1

	@property
	def output_feature_count(self) -> int:
		return 1

	def __call__(self, df: pd.DataFrame) -> "TokenSequenceSequence":
		sequence_groups = df.groupby(
			("CROSS_VALIDATION_ITER", "DYAD", "ROUND", "UTT_START_TIME", "UTT_END_TIME", "ENTITY"), sort=False)
		print("Generating data for {} entity token sequence(s).".format(len(sequence_groups)), file=sys.stderr)
		seq_xy = sequence_groups.apply(self.__create_seq_xy)

		return TokenSequenceSequence(seq_xy)

	def __create_datapoint_x(self, row: pd.Series) -> Tuple[np.ndarray,]:
		# word_features = [0.0] * len(self.__vocab_idxs)
		# The features representing each individual vocabulary word are at the beginning of the feature vector
		# word_features[self.__vocab_idxs[row["WORD"]]] = 1.0
		# word_label = self.label_encoder.transform(row["WORD"])
		word_label = row["WORD_LABEL"]
		# print("Word label: {}".format(word_label), file=sys.stderr)
		# "OneHotEncoder.transform(..)" returns a matrix even if only a single value is passed to it, so get just the first (and only) row
		word_features = self.onehot_encoder.transform(word_label)[0]
		# print("Word features: {}".format(word_features), file=sys.stderr)
		# The word label for the one-hot encoding is that with the same index as the column that has a "1" value, i.e. the highest value in the vector of one-hot encoding values
		# inverse_label = np.argmax(word_features)
		# assert inverse_label == word_label
		# inverse_word = self.label_encoder.inverse_transform([inverse_label])
		# print("Inverse word label: {}".format(inverse_label), file=sys.stderr)
		is_instructor = 1.0 if row["IS_INSTRUCTOR"] else 0.0
		# is_target = 1.0 if row["IS_TARGET"] else 0.0
		other_features = np.array((is_instructor,))
		# result = word_features + other_features
		result = np.concatenate((word_features, other_features))
		# print("Created a vector of {} features.".format(len(result)), file=sys.stderr)
		# NOTE: Returning a tuple is a hack in order to return an instance of "np.ndarray" from "DataFrame.apply()"
		return result,

	def __create_seq_x_matrix(self, seq_df: pd.DataFrame) -> np.matrix:
		# NOTE: The returned tuples have to be unpacked outside of the "apply(..)" function
		vectors = seq_df.apply(self.__create_datapoint_x, axis=1)
		return np.matrix(tuple(vector[0] for vector in vectors))

	def __create_seq_xy(self, seq_df: pd.DataFrame) -> Tuple[np.matrix, np.ndarray]:
		x = self.__create_seq_x_matrix(seq_df)
		y = seq_df["PROBABILITY"].values
		return x, y


class TokenSequenceSequence(keras.utils.Sequence):
	"""
	A sequence (i.e. less confusingly a dataset) of token sequences, each of which is for a given distinct entity, i.e. possible referent.
	"""

	@staticmethod
	def __group_by_seq_len(seq_xy: pd.Series) -> DefaultDict[int, List[Tuple[np.matrix, np.ndarray]]]:
		result = defaultdict(list)
		for xy in seq_xy:
			seq_len = xy[0].shape[0]
			result[seq_len].append(xy)
		return result

	def __init__(self, seq_xy: pd.Series):
		len_dict = self.__group_by_seq_len(seq_xy)
		print("Created {} batches, one for each unique sequence length.".format(len(len_dict)), file=sys.stderr)
		self.seq_batches_by_len = tuple(len_dict.values())

	def __len__(self) -> int:
		return len(self.seq_batches_by_len)

	def __getitem__(self, idx: int) -> Tuple[np.ndarray, np.ndarray]:
		print("Getting batch idx {}.".format(idx), file=sys.stderr)
		batch = self.seq_batches_by_len[idx]
		seq_x = tuple(x for x, y in batch)
		x = np.asarray(seq_x)
		print("X shape: {}".format(x.shape), file=sys.stderr)
		seq_y = tuple(y for x, y in batch)
		y = np.asarray(seq_y)
		# y = np.expand_dims(y, axis=0)
		print("Y shape: {}".format(y.shape), file=sys.stderr)
		return x, y
    

print("Generating training data token sequences.", file=sys.stderr)
data_generator_factory = DataGeneratorFactory(onehot_encoder)
training_data_generator = data_generator_factory(training_df)

Generating training data token sequences.
Generating data for 291 entity token sequence(s).
Created 30 batches, one for each unique sequence length.


In [91]:
from keras.layers import Dense
from keras.layers import LSTM
from keras.models import Sequential

def create_model(input_feature_count: int, output_feature_count: int) -> Sequential:
	result = Sequential()
	# word_embeddings = Embedding(len(vocab), embedding_vector_length, input_length=max_review_length)
	# model.add(word_embeddings)
	# model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
	# input shape is a pair of (timesteps, features) <https://stackoverflow.com/a/44583784/1391325>
	input_shape = (None, input_feature_count)
	print("Input shape: {}".format(input_shape), file=sys.stderr)
	units = output_feature_count
	print("Units: {}".format(units), file=sys.stderr)
	lstm = LSTM(input_shape=input_shape, units=units, dropout=0.1, recurrent_dropout=0.1)
	# lstm = LSTM(batch_input_shape = training_x.shape, stateful = True, units=len(training_y.shape))
	result.add(lstm)
	result.add(Dense(units, activation='sigmoid'))
	result.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	print(result.summary())
	return result

model = create_model(data_generator_factory.input_feature_count, data_generator_factory.output_feature_count)

Input shape: (None, 250)
Units: 1


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 1)                 1008      
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 2         
Total params: 1,010
Trainable params: 1,010
Non-trainable params: 0
_________________________________________________________________
None


In [93]:
# train LSTM

epochs = 50
print("Training model using {} epoch(s).".format(epochs), file=sys.stderr)
for epoch in range(0, epochs):
    for x, y in training_data_generator:
        print(x)
        print(y)
#        model.fit(x, y, verbose=1)

#fit_generator(self, generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)
training_history = model.fit_generator(training_data_generator, epochs=epochs, verbose=1)

# train LSTM <https://machinelearningmastery.com/memory-in-a-long-short-term-memory-network/>
#epochs = 5
#for i in range(epochs):
#    for seq_idx in range(0, training_x.shape[0]):
        #x = training_x[i]
#        x = training_x[0:31]
 #       x_oldshape = x.shape
 #       x_newshape = (-1, x_oldshape[0], x_oldshape[1])
        #x = np.reshape(x, x_newshape)
        #print("x.shape = {}".format(x.shape), file=sys.stderr)
        #y = training_y[i]
 #       y = training_y[0:31]
 #       y_oldshape = y.shape
 #       y_newshape = (-1, y_oldshape[0])
        #y = np.reshape(y, y_newshape)
        #print("y.shape = {}".format(y.shape), file=sys.stderr)
 #       model.fit(x, y, epochs=1, batch_size=1, verbose=1, shuffle=False)
 #       model.reset_states()
    
    

Training model using 50 epoch(s).
Getting batch idx 0.
X shape: (40, 1, 250)
Y shape: (40, 1)
Getting batch idx 1.
X shape: (34, 2, 250)
Y shape: (34, 2)
Getting batch idx 2.
X shape: (31, 3, 250)
Y shape: (31, 3)
Getting batch idx 3.
X shape: (27, 4, 250)
Y shape: (27, 4)
Getting batch idx 4.
X shape: (23, 5, 250)
Y shape: (23, 5)
Getting batch idx 5.
X shape: (15, 6, 250)
Y shape: (15, 6)
Getting batch idx 6.
X shape: (16, 7, 250)
Y shape: (16, 7)
Getting batch idx 7.
X shape: (13, 8, 250)
Y shape: (13, 8)
Getting batch idx 8.
X shape: (12, 9, 250)
Y shape: (12, 9)
Getting batch idx 9.
X shape: (6, 10, 250)
Y shape: (6, 10)
Getting batch idx 10.
X shape: (13, 11, 250)
Y shape: (13, 11)
Getting batch idx 11.
X shape: (6, 12, 250)
Y shape: (6, 12)
Getting batch idx 12.
X shape: (10, 13, 250)
Y shape: (10, 13)
Getting batch idx 13.
X shape: (9, 14, 250)
Y shape: (9, 14)
Getting batch idx 14.
X shape: (8, 15, 250)
Y shape: (8, 15)
Getting batch idx 15.
X shape: (2, 16, 250)
Y shape: (2, 

[[[ 0.  0.  0. ...,  0.  0.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]]

 ..., 
 [[ 0.  0.  0. ...,  0.  0.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]]]
[[ 0.5137191 ]
 [ 0.5137191 ]
 [ 0.5647695 ]
 [ 0.44514291]
 [ 0.44028732]
 [ 0.38828752]
 [ 0.47564182]
 [ 0.71831793]
 [ 0.58049927]
 [ 0.52894132]
 [ 0.52894132]
 [ 0.45475994]
 [ 0.597051  ]
 [ 0.52220411]
 [ 0.48243448]
 [ 0.74770382]
 [ 0.51527117]
 [ 0.60047067]
 [ 0.57118331]
 [ 0.5725783 ]
 [ 0.54496885]
 [ 0.47222936]
 [ 0.9981083 ]
 [ 0.56881635]
 [ 0.51571733]
 [ 0.99950667]
 [ 0.60098745]
 [ 0.58072271]
 [ 0.68307409]
 [ 0.41307047]
 [ 0.53400748]
 [ 0.56109571]
 [ 0.48225267]
 [ 0.5240197 ]
 [ 0.50188718]
 [ 0.35394939]
 [ 0.76189011]
 [ 0.55248304]
 [ 0.44252374]
 [ 0.49666817]]
[[[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  1

Getting batch idx 26.
X shape: (1, 33, 250)
Y shape: (1, 33)
Getting batch idx 27.
X shape: (1, 34, 250)
Y shape: (1, 34)
Getting batch idx 28.
X shape: (1, 35, 250)
Y shape: (1, 35)
Getting batch idx 29.
X shape: (1, 40, 250)
Y shape: (1, 40)
Getting batch idx 30.
Getting batch idx 0.
X shape: (40, 1, 250)
Y shape: (40, 1)
Getting batch idx 1.
X shape: (34, 2, 250)
Y shape: (34, 2)
Getting batch idx 2.
X shape: (31, 3, 250)
Y shape: (31, 3)
Getting batch idx 3.
X shape: (27, 4, 250)
Y shape: (27, 4)
Getting batch idx 4.
X shape: (23, 5, 250)
Y shape: (23, 5)
Getting batch idx 5.
X shape: (15, 6, 250)
Y shape: (15, 6)
Getting batch idx 6.
X shape: (16, 7, 250)
Y shape: (16, 7)
Getting batch idx 7.
X shape: (13, 8, 250)
Y shape: (13, 8)
Getting batch idx 8.
X shape: (12, 9, 250)
Y shape: (12, 9)
Getting batch idx 9.
X shape: (6, 10, 250)
Y shape: (6, 10)
Getting batch idx 10.
X shape: (13, 11, 250)
Y shape: (13, 11)
Getting batch idx 11.
X shape: (6, 12, 250)
Y shape: (6, 12)
Getting ba

[[[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  1. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]]]
[[  3.03693877e-12   4.52948606e-01   4.86254507e-01   4.92247795e-01
    4.91550637e-01   5.20068058e-01   4.08566377e-01   9.62644629e-01
    4.92247795e-01   4.91550637e-01   5.92635655e-37   5.20068058e-01
    4.08566377e-01   5.73953647e-01   5.73953647e-01   4.92247795e-01
    4.91550637e-01   5.20068058e-01   1.69099814e-12   5.18729758e-01
    4.86254507e-01   4.92247795e-01   4.91550637e-01   5.20068058e-01
    4.08566377e-01   9.62644629e-01   5.73953647e-01   4.52596671e-01
    4.91550637e-01   5.92635655e-37]]
[[[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  1. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]]]
[[  4.56339955e-001   3.72811243e-001   4.59172956e-001 

Getting batch idx 22.
X shape: (4, 23, 250)
Y shape: (4, 23)
Getting batch idx 23.
X shape: (1, 24, 250)
Y shape: (1, 24)
Getting batch idx 24.
X shape: (2, 26, 250)
Y shape: (2, 26)
Getting batch idx 25.
X shape: (1, 30, 250)
Y shape: (1, 30)
Getting batch idx 26.
X shape: (1, 33, 250)
Y shape: (1, 33)
Getting batch idx 27.
X shape: (1, 34, 250)
Y shape: (1, 34)
Getting batch idx 28.
X shape: (1, 35, 250)
Y shape: (1, 35)
Getting batch idx 29.
X shape: (1, 40, 250)
Y shape: (1, 40)
Getting batch idx 30.
Getting batch idx 0.
X shape: (40, 1, 250)
Y shape: (40, 1)
Getting batch idx 1.
X shape: (34, 2, 250)
Y shape: (34, 2)
Getting batch idx 2.
X shape: (31, 3, 250)
Y shape: (31, 3)
Getting batch idx 3.
X shape: (27, 4, 250)
Y shape: (27, 4)
Getting batch idx 4.
X shape: (23, 5, 250)
Y shape: (23, 5)
Getting batch idx 5.
X shape: (15, 6, 250)
Y shape: (15, 6)
Getting batch idx 6.
X shape: (16, 7, 250)
Y shape: (16, 7)
Getting batch idx 7.
X shape: (13, 8, 250)
Y shape: (13, 8)
Getting ba

...


[[  4.80182071e-01   4.63707334e-01   4.86176351e-01   6.59745331e-01
    5.65911737e-01   4.86176351e-01   8.13125244e-02]
 [  4.32053882e-01   4.55494226e-01   4.71513972e-01   4.83480753e-01
    6.38685347e-01   6.62242476e-01   9.25754605e-01]
 [  3.29337236e-01   4.26283785e-01   7.58757578e-01   8.21363533e-01
    4.26283785e-01   8.51784065e-01   8.21363533e-01]
 [  4.22967226e-01   4.35267921e-01   5.24409102e-01   4.50824018e-01
    9.35971457e-01   4.50824018e-01   6.64303827e-01]
 [  5.00161184e-01   4.14952181e-01   3.79360579e-01   2.53914244e-01
    2.54333479e-01   5.07458433e-01   4.98352098e-01]
 [  4.50028094e-01   5.37052571e-01   6.23672872e-01   3.49859567e-01
    4.13896835e-01   7.55355619e-01   0.00000000e+00]
 [  4.75489919e-01   7.40345487e-01   7.68149887e-01   5.60813800e-01
    5.37151653e-01   4.75489919e-01   6.26318244e-01]
 [  4.61182679e-01   4.37469880e-01   4.37469880e-01   3.15349535e-15
    4.61182679e-01   8.68055909e-01   5.35297113e-01]
 [  4.8

X shape: (4, 18, 250)
Y shape: (4, 18)
Getting batch idx 18.
X shape: (4, 19, 250)
Y shape: (4, 19)
Getting batch idx 19.
X shape: (2, 20, 250)
Y shape: (2, 20)
Getting batch idx 20.
X shape: (1, 21, 250)
Y shape: (1, 21)
Getting batch idx 21.
X shape: (2, 22, 250)
Y shape: (2, 22)
Getting batch idx 22.
X shape: (4, 23, 250)
Y shape: (4, 23)
Getting batch idx 23.
X shape: (1, 24, 250)
Y shape: (1, 24)
Getting batch idx 24.
X shape: (2, 26, 250)
Y shape: (2, 26)
Getting batch idx 25.
X shape: (1, 30, 250)
Y shape: (1, 30)
Getting batch idx 26.
X shape: (1, 33, 250)
Y shape: (1, 33)
Getting batch idx 27.
X shape: (1, 34, 250)
Y shape: (1, 34)
Getting batch idx 28.
X shape: (1, 35, 250)
Y shape: (1, 35)
Getting batch idx 29.
X shape: (1, 40, 250)
Y shape: (1, 40)
Getting batch idx 30.
Getting batch idx 0.
X shape: (40, 1, 250)
Y shape: (40, 1)
Getting batch idx 1.
X shape: (34, 2, 250)
Y shape: (34, 2)
Getting batch idx 2.
X shape: (31, 3, 250)
Y shape: (31, 3)
Getting batch idx 3.
X shap

[[[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]]

 ..., 
 [[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]]

 [[ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]
  [ 0.  0.  0. ...,  0.  0.  1.]]]
[[  6.29178798e-001   4.48419794e-001   4.90061338e-001   3.47877611e-015]
 [  4.47614774e-001   4.84389616e-001   5.74961554e-001   6.79124512e-001]
 [  5.97050998e-001   4.67139113e-001  

X shape: (13, 8, 250)
Y shape: (13, 8)
Getting batch idx 8.
X shape: (12, 9, 250)
Y shape: (12, 9)
Getting batch idx 9.
X shape: (6, 10, 250)
Y shape: (6, 10)
Getting batch idx 10.
X shape: (13, 11, 250)
Y shape: (13, 11)
Getting batch idx 11.
X shape: (6, 12, 250)
Y shape: (6, 12)
Getting batch idx 12.
X shape: (10, 13, 250)
Y shape: (10, 13)
Getting batch idx 13.
X shape: (9, 14, 250)
Y shape: (9, 14)
Getting batch idx 14.
X shape: (8, 15, 250)
Y shape: (8, 15)
Getting batch idx 15.
X shape: (2, 16, 250)
Y shape: (2, 16)
Getting batch idx 16.
X shape: (1, 17, 250)
Y shape: (1, 17)
Getting batch idx 17.
X shape: (4, 18, 250)
Y shape: (4, 18)
Getting batch idx 18.
X shape: (4, 19, 250)
Y shape: (4, 19)
Getting batch idx 19.
X shape: (2, 20, 250)
Y shape: (2, 20)
Getting batch idx 20.
X shape: (1, 21, 250)
Y shape: (1, 21)
Getting batch idx 21.
X shape: (2, 22, 250)
Y shape: (2, 22)
Getting batch idx 22.
X shape: (4, 23, 250)
Y shape: (4, 23)
Getting batch idx 23.
X shape: (1, 24, 250)


ValueError: Error when checking target: expected dense_18 to have shape (None, 1) but got array with shape (10, 13)

In [9]:
import matplotlib.pyplot as plt

# https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

# list all data in history
#print(training_history.history.keys())
# summarize history for accuracy
plt.plot(training_history.history['acc'])
plt.plot(training_history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(training_history.history['loss'])
plt.plot(training_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [28]:
# test LSTM

import logging

def onehot_encodings(features : np.ndarray, onehot_encoder) -> np.ndarray:
    """
    :param features: A 2D array of feature vectors, each of which starts with one-hot encoding features. Any other features follow those.
    :param onehot_encoder: The instance used to encode the original values.
    :return: A 2D numpy array consisting only of one-hot encoding features.
    """
    feature_count = onehot_encoder.n_values_[0]
    return features[:,:feature_count + 1]

def onehot_encoded_word(onehot_encodings : np.ndarray, label_encoder) -> str:
    # Check if there are any non-zero values
    if onehot_encodings.any():
        word_label = onehot_encodings.argmax()
        result = label_encoder.inverse_transform([word_label])[0]
    else:
        result = "(padding)"
    return result

def word(features : np.ndarray, label_encoder, onehot_encoder) -> str:
    feature_count = onehot_encoder.n_values_[0]
    onehot = features[:feature_count + 1]
    return onehot_encoded_word(onehot, label_encoder)

def word_seq(x : np.ndarray, label_encoder, onehot_encoder) -> np.ndarray:
    word_features = onehot_encodings(x, onehot_encoder)
    return np.apply_along_axis(lambda arr : onehot_encoded_word(arr, label_encoder), 1, word_features)

def create_padded_seq_matrix(group: pd.DataFrame, seq_matrix_factory):
    seq_matrices = tuple(seq_matrix_factory(group))
    print("Created a dataset composed of {} sequence(s), with a max sequence length of {}.".format(len(seq_matrices), max(m.shape[0] for m in seq_matrices)), file=sys.stderr)
    print("Padding data sequences.", file=sys.stderr)
    return keras.preprocessing.sequence.pad_sequences(seq_matrices, maxlen=None, padding='pre', truncating='pre', value=0.0)
    

def test_token_seq(token_seq_group : pd.DataFrame, seq_matrix_factory, model: Sequential, label_encoder, onehot_encoder): 
    entity_groups = token_seq_group.groupby("ENTITY", as_index=False, sort=False)
    entity_matrices = tuple(entity_groups.apply(lambda group : (group.name, seq_matrix_factory(group))))
    #for _, m in entity_matrices:
    #    y = m[:, -1]
    #    print(y) 
    
    logging.debug("Created matrices for %d entities, with a max sequence length of %d.", len(entity_matrices), max(m.shape[0] for _, m in entity_matrices))
    logging.debug("Padding entity test data sequences.")
    test_batch = keras.preprocessing.sequence.pad_sequences(tuple(m for _, m in entity_matrices), maxlen=None, padding='pre', value=0.0, dtype='int32')
    print(test_batch[:, :, :-1])
    logging.debug("Test batch shape: %s", test_batch.shape)
    test_x, test_y = split_xy(test_batch)
    
    
    for group_name, m in entity_matrices:
        m = np.expand_dims(m, axis=0)
        print(m.shape)
        x = m[:, :, :-1]
        print(x.shape)
        y = m[:, :, -1]
        print(y.shape)
        model.predict(x, verbose=0, batch_size=1)
    
    #logging.debug("Test X shape: %s", test_x.shape)
    #logging.debug("Test Y shape: %s", test_y.shape)
    # Classify all sets of datapoints for each entity as a single batch
    #entity_predicted_scores = model.predict(test_x, verbose=0)
    
    max_score = 0
    for idx in range(0, test_x.shape[0]):
        entity_id = entity_matrices[idx][0]
        # Get the scores for each entity
        entity_x = test_x[idx]
        logging.debug("Re-scoring token sequence for entity %s: %s", entity_id, word_seq(entity_x, label_encoder, onehot_encoder))
        entity_orig_scores = test_y[idx]
        for tok_idx, features in enumerate(entity_x):
            w = word(features, label_encoder, onehot_encoder)
            s = entity_orig_scores[tok_idx]
            print("{}; Orig score: {}".format(w, s))
        
        max_score = max(max_score, entity_orig_scores.argmax())
        #print(entity_orig_scores)
        orig_score = entity_orig_scores.sum()
        #predicted_scores = entity_predicted_scores[idx]
        #print(predicted_scores)
        #rescored = np.dot(entity_orig_scores, predicted_scores)
        #logging.warning("Original score: %f; New score: %f", orig_score, rescored)
                        
    print("Max score: {}".format(max_score))
            
token_seq_groups = test_df.groupby(
                ("CROSS_VALIDATION_ITER", "DYAD", "ROUND", "UTT_START_TIME", "UTT_END_TIME"),
                as_index=False, sort=False)
print("Will test {} token sequences.".format(len(token_seq_groups)), file=sys.stderr)
test_results = token_seq_groups.apply(lambda group: test_token_seq(group, seq_matrix_factory, model, label_encoder, onehot_encoder))

                        
#test_x, test_y = split_xy(test_matrix)
#print("Training X shape: {}".format(test_x.shape), file=sys.stderr)
#print("Training Y shape: {}".format(test_y.shape), file=sys.stderr)

#seq_predicted_values = model.predict(test_x, verbose=0)
#model.reset_states()
#print("result.shape = {}".format(seq_predicted_values.shape), file=sys.stderr)
#for i, tested_seq in enumerate(test_x):
#    word_features = onehot_encodings(tested_seq, onehot_encoder)
    #print("word features: {}".format(word_features))
    #print("Tested sequence: {}".format(tested_seq))
    #print("Tested sequence shape: {}".format(tested_seq.shape))
    #seq_labels = word_features.argmax(axis=1)
    #print("Inverse labels: {}".format(inverse_labels))
    #seq_words = label_encoder.inverse_transform(seq_labels)
#    seq_words = np.apply_along_axis(lambda arr : onehot_encoded_word(arr, label_encoder), 1, word_features)
    #print("Inverse word labels: {}".format(seq_words))
#    predicted_values = seq_predicted_values[i]
#    assert tested_seq.shape[:-1] == predicted_values.shape
    #assert tested_seq.shape[:-1] == actual_values.shape
    #rint("Predicted values: {}".format(predicted_values))
#    actual_values = test_y[i]
#    assert predicted_values.shape == actual_values.shape
    
#    differences = predicted_values - actual_values
#    print("[Predicted_val - Actual_val]: {}".format(differences))
    #print("Actual values: {}".format(actual_values))
	#print('X=%.1f y=%.1f, yhat=%.1f' % (seq1[i], seq1[i+1], result[i]))

[[[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]

 [[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]

 [[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]

 ..., 
 [[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]

 [[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]

 [[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]]
(1, 3, 251)
(1, 3, 250)
(1, 3)
[[[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]

 [[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]

 [[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]

 ..., 
 [[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]

 [[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]

 [[0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]
  [0 0 0 ..., 0 0 1]]]
(1, 3, 251)
(1, 3, 250)
(1, 3)


Will test 125 token sequences.


ValueError: Error when checking : expected lstm_1_input to have shape (None, 40, 250) but got array with shape (1, 3, 250)